# astroHACK VLA Tutorial




## Download Tutorial Data

In [ ]:
from astrohack.gdown_utils import gdown_data
gdown_data('ea25_cal_small_spw1_4_60_ea04_after.ms',download_folder='data')

## Setup Dask Local Cluster

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

client = astrohack_local_client(cores=2,memory_limit='8GB')
client

## Extract Holog

In [2]:
from astrohack.extract_holog import extract_holog
scans=[8,9,10,12,13,14,16,17,18,23,24,25,27,28,29,31,32,33,38,39,40,42,43,44,46,47,48,53,54,55,57]
holog_obs_description = {'map_0' :{'scans':scans,'ant':{'ea25':['ea04']}}} #change 0 -> map_0
holog_obs_description['ddi'] = [0]

holog_mds = extract_holog(ms_name='data/ea25_cal_small_spw1_4_60_ea04_after.ms', 
                                     holog_obs_dict=holog_obs_description,
                                     data_col='CORRECTED_DATA',
                                     parallel=True,
                                     overwrite=True,
                                    )

data/ea25_cal_small_spw1_4_60_ea04_after.holog.zarr will be overwritten.
data/ea25_cal_small_spw1_4_60_ea04_after.point.zarr will be overwritten.


Successful readonly open of usernoread-locked table data/ea25_cal_small_spw1_4_60_ea04_after.ms/ANTENNA: 8 columns, 27 rows
Successful readonly open of usernoread-locked table data/ea25_cal_small_spw1_4_60_ea04_after.ms: 23 columns, 9145 rows
Successful readonly open of default-locked table data/ea25_cal_small_spw1_4_60_ea04_after.ms/ANTENNA: 8 columns, 27 rows
Successful readonly open of default-locked table data/ea25_cal_small_spw1_4_60_ea04_after.ms/OBSERVATION: 9 columns, 1 rows


The holog_mds object is a python dict that has extended with the following functionality 

In [7]:
holog_mds.summary()

+-------+-------+--------------+
| ddi   | scan  | antenna      |
+-------+-------+--------------+
| ddi_0 | map_0 | ['ant_ea25'] |
+-------+-------+--------------+
{
        "ddi_0": {
                "map_0": {
                        "ant_ea25": "<xarray.Dataset>\nDimensions:              (time: 9145, lm: 2, chan: 57, pol: 4)\nCoordinates:\n  * chan                 (chan) float64 1.411e+10 1.411e+10 ... 1.422e+10\n  * pol                  (pol) <U2 'RR' 'RL' 'LR' 'LL'\n  * time                 (time) float64 5.17e+09 5.17e+09 ... 5.17e+09 5.17e+09\nDimensions without coordinates: lm\nData variables:\n    DIRECTIONAL_COSINES  (time, lm) float64 -0.004335 -0.002795 ... 0.009569\n    VIS                  (time, chan, pol) complex64 0j ... (0.008969664-0.01...\n    WEIGHT               (time, chan, pol) float64 0.0 0.0 ... 6.619e+04\nAttributes:\n    antenna_name:         ea25\n    ddi:                  0\n    holog_map_key:        map_0\n    l_max:                0.009673800837874147\n

In [10]:
holog_mds['ddi_0']['map_0']['ant_ea25']

<xarray.Dataset>
Dimensions:              (time: 9145, lm: 2, chan: 57, pol: 4)
Coordinates:
  * chan                 (chan) float64 1.411e+10 1.411e+10 ... 1.422e+10
  * pol                  (pol) <U2 'RR' 'RL' 'LR' 'LL'
  * time                 (time) float64 5.17e+09 5.17e+09 ... 5.17e+09 5.17e+09
Dimensions without coordinates: lm
Data variables:
    DIRECTIONAL_COSINES  (time, lm) float64 -0.004335 -0.002795 ... 0.009569
    VIS                  (time, chan, pol) complex64 0j ... (0.008969664-0.01...
    WEIGHT               (time, chan, pol) float64 0.0 0.0 ... 6.619e+04
Attributes:
    antenna_name:         ea25
    ddi:                  0
    holog_map_key:        map_0
    l_max:                0.009673800837874147
    l_min:                -0.009671789717177779
    m_max:                0.009698335330087179
    m_min:                -0.009771580421317516
    parallactic_samples:  [5.307972218541546, 5.354953165003047, 5.4937719298...
    telescope_name:       EVLA

## Holog

In [ ]:
from astrohack import holog
import numpy as np

cell_size = np.array([-0.0006442, 0.0006442]) #arcseconds
grid_size = np.array([31, 31]) #pixels

image_mds = holog(holog_name='data/ea25_cal_small_spw1_4_60_ea04_after.holog.zarr',
                  grid_size=grid_size, cell_size=cell_size, 
                  overwrite=True,
                  phase_fit=True,
                  apply_mask=True,
                  to_stokes=True,
                  parallel=True
                 )

In [ ]:
image_mds.summary()

In [ ]:
image_mds['ant_ea25']['ddi_0']

In [ ]:
image_mds['ant_ea25']['ddi_0'].ANGLE.isel(chan=0,pol=0).plot()

## Panel

In [ ]:
from astrohack.panel import panel
panel_model = 'rigid'

panel_mds = panel(
                    image_name='data/ea25_cal_small_spw1_4_60_ea04_after.image.zarr', 
                    panel_model=panel_model,
                    unit='mils', 
                    parallel=True,
                    overwrite=True
                )

panel_mds

In [19]:
a.summary()

+----------+-----------+
| antenna  | ddi       |
+----------+-----------+
| ant_ea25 | ['ddi_0'] |
+----------+-----------+


In [20]:
a['ant_ea25']['ddi_0']

<xarray.Dataset>
Dimensions:             (u: 442, v: 442, labels: 172, pars: 3, screws: 4)
Coordinates:
  * labels              (labels) object '1- 1' '1- 2' '1- 3' ... '6-39' '6-40'
  * pars                (pars) int64 0 1 2
  * screws              (screws) <U2 'il' 'ir' 'ol' 'or'
  * u                   (u) float64 13.72 13.66 13.6 ... -13.54 -13.6 -13.66
  * v                   (v) float64 -13.72 -13.66 -13.6 ... 13.54 13.6 13.66
Data variables:
    AMPLITUDE           (u, v) float64 dask.array<chunksize=(221, 221), meta=np.ndarray>
    CORRECTIONS         (u, v) float64 dask.array<chunksize=(221, 221), meta=np.ndarray>
    DEVIATION           (u, v) float64 dask.array<chunksize=(221, 221), meta=np.ndarray>
    MASK                (u, v) bool dask.array<chunksize=(442, 442), meta=np.ndarray>
    PANEL_DISTRIBUTION  (u, v) int32 dask.array<chunksize=(221, 221), meta=np.ndarray>
    PANEL_PARAMETERS    (labels, pars) float64 dask.array<chunksize=(172, 3), meta=np.ndarray>
    PANEL_SCREWS        (labels, screws) float64 dask.array<chunksize=(172, 4), meta=np.ndarray>
    PHASE               (u, v) float64 dask.array<chunksize=(221, 221), meta=np.ndarray>
    PHASE_CORRECTIONS   (u, v) float64 dask.array<chunksize=(221, 221), meta=np.ndarray>
    PHASE_RESIDUALS     (u, v) float64 dask.array<chunksize=(221, 221), meta=np.ndarray>
    RESIDUALS           (u, v) float64 dask.array<chunksize=(221, 221), meta=np.ndarray>
Attributes: (12/13)
    AIPS:              False
    amp_unit:          V
    antenna_name:      ea25
    cutoff:            0.7775090739057724
    input_gain:        99.47131763207551
    input_rms:         0.0004844531604180805
    ...                ...
    output_rms:        0.00022835994354467628
    panel_kind:        rigid
    panel_margin:      0.2
    telescope_name:    VLA
    theoretical_gain:  99.60312188541826
    wavelength:        0.0211598290513834